In [7]:
import json
from datetime import datetime
import requests
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from twilio.rest import Client
from scripts_config import TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, PHONE_NUMBER_SMS, PHONE_NUMBER_WHATSAPP, EXCHANGE_TOKEN
import pandas as pd
import numpy as np

In [2]:
#------------------------------------OBTENER VALOR DEL DOLLAR HOY-------------------------------
#REQUEST
url_dollar = "https://api.apilayer.com/exchangerates_data/convert?to=COP&from=USD&amount=1"

payload = {}
headers= {
  "apikey": EXCHANGE_TOKEN #Key del API
}

response = requests.request("GET", url_dollar, headers=headers, data = payload)

status_code = response.status_code
result = response.json()

In [3]:
#Para json anidados ponga json[primerallave][segundallave]
#print(result['query']['from'])
price_dollar = result['result']
print(price_dollar)

4683.2


In [4]:
#------------------------------------OBTENER EL LISTADO DE CRIPTOS-------------------------------
url_criptos = 'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&ids=bitcoin%2Cethereum%2Cdogecoin%2Ctether%2Ccardano%2Ctron%2Cchainlink%2Ceos%2Cdai%2Cfrax&order=market_cap_desc&per_page=100&page=1&sparkline=false&price_change_percentage=24'
df_criptos = pd.read_json(url_criptos)
#Select a los campos que requerimos
df_criptos = df_criptos[['id', 'symbol', 'name', 'current_price', 'high_24h', 'low_24h', 'price_change_24h', 'price_change_percentage_24h']]
#renombrar el campo
df_send = df_criptos.rename(columns ={'price_change_percentage_24h':'variation_porcent'})
#Seteamos el indice
df_send = df_send[['name', 'current_price', 'variation_porcent']]
df_send.set_index('name', inplace= True)
df_send
#-----------------------------------------------------------------


,current_price,variation_porcent
name,,
Bitcoin,21093.000000,1.51742
Ethereum,1556.480000,2.09914
Tether,1.002000,0.17703
Cardano,0.337828,1.95342
Dogecoin,0.081297,0.51174
TRON,0.060701,2.09357
Dai,1.000000,-0.06441
Chainlink,6.490000,1.74496
EOS,0.978054,0.68534


In [11]:
#------------------SMS------------------------
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN


client = Client(account_sid, auth_token)
#Mensaje a mi celular
message = client.messages.create(
    body='\n¡Hola! \n \n El valor del dolar hoy en COP es: '+str(price_dollar)+' \n \n  El precio de la cripto divisa es: \n \n ' + str(df_send),
    from_=PHONE_NUMBER,
    to='+573228784083'
)
print(message.sid)

SMd157f7830a30d6f52853c87219c26722


In [6]:
#-------------WHATSAPP-------------
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN
client = Client(account_sid, auth_token)

message = client.messages.create(
                              body='\n¡Hola! \n \n El valor del dolar hoy en COP es: '+str(price_dollar)+' \n \n  El precio de la cripto divisa es: \n \n ' + str(df_send),
                              from_='whatsapp:'+PHONE_NUMBER_WHATSAPP,
                              to='whatsapp:+573228784083'
                          )
print(message.sid)

SMc8f3788436371c04e692a9645aec0212
